In [2]:
from rs_graph.data import load_basic_repos_dataset

repos = load_basic_repos_dataset().sample(400)
repos

,repo,title,doi,published_date
1332,https://github.com/Nicolas-Schmidt/BayesMFSurv,BayesMFSurv: An R Package to Estimate Bayesian...,10.21105/joss.02164,2020-04-24T19:42:40.868Z
1952,https://github.com/juliohm/GeoStats.jl,GeoStats.jl -- High-performance geostatistics ...,10.21105/joss.00692,2018-04-25T10:27:20.366Z
527,https://github.com/kplabs-pl/OXI,OXI: An online tool for visualization and anno...,10.1016/j.softx.2023.101476,2023-07-31
762,https://github.com/llrs/experDesign,experDesign: stratifying samples into batches ...,10.21105/joss.03358,2021-11-27T08:31:41.851Z
574,https://github.com/chrisaberson/BetterReg,BetterReg: An R package for Useful Regression ...,10.21105/joss.04280,2022-06-02T17:30:41.437Z
...,...,...,...,...
136,https://github.com/mathLab/PINA,Physics-Informed Neural networks for Advanced ...,10.21105/joss.05352,2023-07-19T17:41:02.862Z
1653,https://www.github.com/singularityhub/containe...,Container Tree: Software to Model Container Fi...,10.21105/joss.01418,2019-05-19T20:32:55.927Z
342,https://github.com/schochastics/signnet,signnet: An R package for analyzing signed net...,10.21105/joss.04987,2023-01-27T10:04:07.320Z
2102,https://github.com/barbagroup/cuIBM,cuIBM: a GPU-based immersed boundary method code,10.21105/joss.00301,2017-07-21T00:00:00.000Z


In [13]:
# for each repo, get the reademe (if it exists) and the description.
from dotenv import load_dotenv
from ghapi.all import GhApi
from rs_graph.data.enrichment.github import get_repo_parts_from_url
import base64

load_dotenv()

api = GhApi()

def get_repo_readme(repo: str) -> str | None:
    repo_parts = get_repo_parts_from_url(repo)
    try:
        b64_encoded_readme = api.repos.get_readme(
                owner=repo_parts.owner,
                repo=repo_parts.repo,
            )["content"]
        
        return base64.b64decode(b64_encoded_readme).decode("utf-8")
    except:
        return None

def get_repo_description(repo: str) -> str | None:
    repo_parts = get_repo_parts_from_url(repo)
    try:
        return api.repos.get(
                owner=repo_parts.owner,
                repo=repo_parts.repo,
            )["description"]
    except:
        return None

get_repo_description("https://github.com/evamaxfield/rs-graph")

'Research Software Graph'

'# Research Software Graph\n\n🚀 [Rendered Interactive Graph](https://evamaxfield.github.io/rs-graph/) 🚀\n\nGraphing the dependencies (and dependents) of research software and their contributors.\n\n## Setup\n\n* Install [Just](https://github.com/casey/just#packages)\n* Create a new Conda / Mamba Env (Python 3.11 preferred)\n* Install lib: `just install` (or `pip install .`)\n\n### Data Download\n\nThere is a lot of pre-processed data stored on a GCS bucket. To download it, run:\n\n```bash\nrs-graph-data download\n```\n\nThis will always download the latest version of the data (as pushed by @evamaxfield).\n\n## Order of Data Operations\n\nIf this project was starting from scratch, the order of operations to recreate our work\nwould be:\n\n**Note:** at each step of the process, the data or model is cached\nto the installed package directory (`rs_graph.data.files`).\n\n1. Get all data sources using the command: `rs-graph-sources get-all`\n2. Options\n    1. Get all recursive upstream depe